<a href="https://colab.research.google.com/github/GirolamoOddo/AppliedMath_Notebooks/blob/main/PySINDy_for_ControlledSystemIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview PySINDy Colab   
Application of pysindy in colab environment, here you can find:


*   Train and Test SINDy Model w/o control inputs
*   Train and Test SINDy Model w/ control inputs and
    custom library

This notebook is self-contained and can work with either synthetic or imported data.  
Please note the following core dependencies:  
>numpy: 1.23.5  
>pysindy: 1.7.5  
>pandas: 1.5.3

For each case, there is also a plot section that highlights both the system and, if applicable, the control inputs.



USEFUL RESOURCES:

* OFFICIAL GIT: https://github.com/dynamicslab/pysindy  



* OFFICIAL DOC: https://pysindy.readthedocs.io/en/latest/index.html  

* UNOFFICIAL NOTES: https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/DL2/Dynamical_Neural_Networks/Complete_DNN_2_1.html?highlight=PySINDy

REFERENCES:

* Brian M. de Silva, Kathleen Champion, Markus Quade, Jean-Christophe Loiseau, J. Nathan Kutz, and Steven L. Brunton., (2020). PySINDy: A Python package for the sparse identification of nonlinear dynamical systems from data. Journal of Open Source Software, 5(49), 2104, https://doi.org/10.21105/joss.02104

* Kaptanoglu et al., (2022). PySINDy: A comprehensive Python package for robust sparse system identification. Journal of Open Source Software, 7(69), 3994, https://doi.org/10.21105/joss.03994







  
  
---

# SINDy Download and Data Acqusition

In [ ]:
# @title Import SINDy (mandatory)

%%capture

!rm -rf pysindy
!rm -rf sample_data

!git clone https://github.com/dynamicslab/pysindy.git
!cd pysindy/pysindy/.
!ls
!pip install pysindy


In [ ]:
# @title Import Utilities (mandatory)
import warnings
import numpy as np
import pandas as pd
import pysindy as ps
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from sklearn.linear_model import Lasso
from mpl_toolkits.mplot3d import Axes3D
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, IdentityLibrary


#_______________________________________________________________________________
# CHECK PACKAGES VERSION:
#_______________________________________________________________________________
#import os
#packages = [
#    'numpy',
#    'pysindy',
#    'pandas',
#    'plotly'
#    'scipy',
#    'sklearn'
#]
#
#for package in packages:
#    try:
#        version = os.popen(f"pip show {package} | grep Version").read().strip().split(': ')[1]
#        print(f"{package}: {version}")
#    except Exception as e:
#        print(f"Error getting version for {package}: {e}")
#
#_______________________________________________________________________________
# THE FOLLOWING CODE WORKS PROPERLY WITH THESE PACKAGES VERSION:
#_______________________________________________________________________________
#numpy: 1.23.5
#pysindy: 1.7.5
#pandas: 1.5.3
#_______________________________________________________________________________

Now choose whether to: Load your own data or Generate a synthetic dataset.

In [ ]:
# @title Load your own data (if you have it)
# Import custom data

#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)

# Correct the path for your Drive
#train_data_path = '/content/drive/MyDrive/colab_notebooks/colab_set/colab_train'
#test_data_path = '/content/drive/MyDrive/colab_notebooks/colab_set/colab_test'

#x_train = pd.read_csv(train_data_path+"train_sindy.csv")
#x_test = pd.read_csv(test_data_path+"test_sindy.csv")
#dt = 0.01


In [ ]:
# @title Generate sintetic data (recommended option)
# Simulate data
dt = 0.00005
time = np.arange(0, 20, dt)
time_test = np.arange(0, 4, dt)
coeff = np.array([1, 1, 1])

# All 'Train data' are editable
# Train data: Variables
x = np.sin(time*4) + coeff[0]
x = x.reshape(-1, 1)

y = np.cos(time*4) + coeff[1]
y = y.reshape(-1, 1)

z = np.exp(-time) + coeff[2]
z = z.reshape(-1, 1)

x0 = np.array([x[0], y[0], z[0]])

x_train = np.column_stack((x, y, z))
x_train_norm = x_train / x_train.max(axis=0)

# Train data: Controls
u0 = time**(1/3) - np.sin(time)
u0 = u0.reshape(-1, 1)

u1 = time
u1 = u1.reshape(-1, 1)

u_train_control = np.column_stack((u0, u1))
u_train_norm_control = u_train_control / u_train_control.max(axis=0)


# Test data: Controls, Same Control Strategy
u0_test = time_test**(1/3) - np.sin(time_test)
u0_test = u0_test.reshape(-1, 1)

u1_test = time_test
u1_test = u1_test.reshape(-1, 1)

u_test_control = np.column_stack((u0_test, u1_test))
u_test_norm_control = u_test_control / u_test_control.max(axis=0)

# Test data: Controls, Diff. Control Strategy
u0_test_2 = np.exp(time_test)*np.sin(time_test**4)
u0_test_2 = u0_test_2.reshape(-1, 1)

u1_test_2 = np.exp(time_test**3)
u1_test_2 = u1_test_2.reshape(-1, 1)

u_test_control_2 = np.column_stack((u0_test_2, u1_test_2))
u_test_norm_control_2 = u_test_control_2 / u_test_control_2.max(axis=0)

#_____________________________________________________________________


#print(x_train_norm)
#print(u_train_control)

---



# Train and Test SINDy w/o Control Actions

In this section, the dynamic system will be modeled using SINDy, without any control actions. Therefore, the system evolves freely, and SINDy will identify the ODEs governing the system.

In [ ]:
# @title View System Trajectory
trace = go.Scatter3d(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    mode='lines', name='Trajectory', line=dict(color='blue')
)

layout = go.Layout(
    scene=dict(
        bgcolor='black',
         xaxis=dict(range=[-5, 5]),
         yaxis=dict(range=[-5, 5]),
         zaxis=dict(range=[0, 5])),
    margin=dict(l=0, r=0, b=0, t=0))

fig = go.Figure(data=[trace], layout=layout)
fig.show()

  ## Train and Test the SINDy model  
Pay attention to the `threshold` value as it affects the complexity of the resulting system; the higher this threshold value is, the sparser the result will be.

In [ ]:
# @title Train the SINDy model
# Train the model


identity_lib = IdentityLibrary()
poly_lib = PolynomialLibrary(degree=2)
fourier_lib = FourierLibrary( include_sin=True, include_cos=True)
combined_lib = identity_lib+fourier_lib

feature_names = ['x','y','z']
opt = ps.STLSQ(threshold=0.2) #0.2

model = ps.SINDy(differentiation_method=ps.FiniteDifference(order=2), feature_names=feature_names, feature_library=combined_lib, optimizer=opt)
model.fit(x_train, t=dt)
model.print()

(x)' = 4.001 y + -2.842 z + 0.479 sin(1 z) + -2.889 cos(1 z)
(y)' = -4.000 x + 2.838 z + -0.473 sin(1 z) + 2.886 cos(1 z)
(z)' = -0.364 z + 0.672 cos(1 z)


In [ ]:
# @title Test the SINDy model
# Simulate the model
x0_new = x0.flatten()
x_simulated = model.simulate(x0_new, t=time_test, integrator='odeint')

x_sim = x_simulated[:,0]
y_sim = x_simulated[:,1]
z_sim = x_simulated[:,2]


x0_new_2 = np.array([2, 3, 3])
x_simulated_2 = model.simulate(x0_new_2, t=time_test, integrator='odeint')

x_sim_2 = x_simulated_2[:,0]
y_sim_2 = x_simulated_2[:,1]
z_sim_2 = x_simulated_2[:,2]


trace = go.Scatter3d(
    x=x_sim.flatten(), y=y_sim.flatten(), z=z_sim.flatten(),
    mode='lines', name='Sim Trajectory Same I.C.', line=dict(color='red'))

layout = go.Layout(
    scene=dict(
        bgcolor='black',
         xaxis=dict(range=[-5, 5]),
         yaxis=dict(range=[-5, 5]),
         zaxis=dict(range=[0, 5])),
    margin=dict(l=0, r=0, b=0, t=0))

fig = go.Figure(data=[trace], layout=layout)

trace2 = go.Scatter3d(
    x=x_sim_2.flatten(), y=y_sim_2.flatten(), z=z_sim_2.flatten(),
    mode='lines', name='Sim Trajectory Diff I.C.', line=dict(color='green'))

fig = go.Figure(data=[trace], layout=layout)

trace1 = go.Scatter3d(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    mode='lines', name='Training Trajectory', line=dict(color='blue'))

fig.add_trace(trace2)
fig.add_trace(trace1)
fig.show()



---



# Train and Test SINDy w/ Control Actions and Custom Libraries

In this section, we will do the same as in the previous section, but now the considered system will have control actions and will not evolve freely. In the SINDy model we want to obtain, it will be necessary to consider these inputs, which will be identified as 'u'.

In [ ]:
# @title View System Trajectory w/ Control
trace = go.Scatter3d(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    mode='lines', name='Trajectory', line=dict(color='blue'))

layout = go.Layout(
    scene=dict(
         bgcolor='black',
         xaxis=dict(range=[-5, 5]),
         yaxis=dict(range=[-5, 5]),
         zaxis=dict(range=[ 0, 5])),
    margin=dict(l=0, r=0, b=0, t=0))

fig = go.Figure(data=[trace], layout=layout)
fig.show()

plt.style.use('dark_background')

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 6))

ax1.plot(time, u0.flatten(), label='Control Signal 0', color='green')
ax1.set_ylabel('Control Signal 0')
ax1.grid(True)
ax1.legend()

ax2.plot(time, u1.flatten(), label='Control Signal 1', color='red')
ax2.set_xlabel('Time')
ax2.set_ylabel('Control Signal 1')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

## Build a Custom Candidate Library

Here, we create a library by composing default SINDy libraries.  

This part is not mandatory for the development of the SINDy model with control but is included for completeness. Furthermore, it is possible to generalize more and build a custom library from scratch.  

Please note that using very large candidate function libraries tends to make the candidate matrix ill-conditioned or even singular. Therefore, it is advisable not to excessively expand the library unless in specific operational cases.


In [ ]:
# @title Build a Custom Candidate Library Code
# Build a custom library

# Call default SINDy libraries
poly_lib = PolynomialLibrary(degree=2)
fourier_lib = ps.FourierLibrary(include_sin=True, include_cos=True)
identity_lib = ps.IdentityLibrary()

# Custom lib w/ custom function
library_functions = [lambda x: np.tanh(x)]
library_function_names = [
    lambda x: "tanh(" + x + ")"
]
tanh_lib = ps.CustomLibrary(
    library_functions=library_functions,
    function_names=library_function_names
)

# Sum libraries
concat_lib = identity_lib+fourier_lib+poly_lib
# Tensor libraries
tensor_lib = identity_lib*fourier_lib*poly_lib
# Custom library
# NOTE: Using huge custom_lib make candidate matrix ill conditioned or singular
custom_lib = concat_lib+tensor_lib
combined_lib = identity_lib*fourier_lib+identity_lib

 ## Train and Test the SINDy model w/ Control Actions
The difference, compared to the previous code, lies in the availability of vectors with control actions.  
 `model.fit(x_train, u=u_train_control, t=dt)`

In [ ]:
# @title Train the SINDy model w/ Control
# Train the model w/ control inputs
opt_contr = ps.STLSQ(threshold=0.32)

model = ps.SINDy(feature_library=combined_lib,
                 feature_names=['x', 'y', 'z', 'u0', 'u1'],
                 optimizer=opt_contr)
model.fit(x=x_train, u=u_train_control, t=dt)
model.get_feature_names()
model.print()

(x)' = 0.756 y sin(1 z) + 0.366 y cos(1 z) + -2.445 z cos(1 z) + -1.511 u0 sin(1 z) + -1.084 u1 cos(1 z) + 3.165 y + -2.683 z + 1.272 u0 + 0.586 u1
(y)' = -1.752 z sin(1 z) + 1.965 z cos(1 z) + 0.880 u0 sin(1 z) + 0.733 u1 cos(1 z) + -4.000 x + 4.415 z + -0.741 u0 + -0.396 u1
(z)' = -0.352 z sin(1 z) + 0.538 z cos(1 z)


Note that since the control actions are totally invented, the SINDy model finds the low significance and removes them by increasing the sparsity treshold.

In [ ]:
# @title Test the SINDy model w/ Control
# Simulate the model w/ control inputs

# Same IC Same Control Strategy
x0_new_cont = x0.flatten()
x_simulated_cont = model.simulate(x0_new_cont, t=time_test, u=u_test_control, integrator='odeint')

x_sim_cont = x_simulated_cont[:,0]
y_sim_cont = x_simulated_cont[:,1]
z_sim_cont = x_simulated_cont[:,2]

# Diff IC Same Control Strategy
x0_new_cont_2 = np.array([2, 3, 3])
x_simulated_cont_2 = model.simulate(x0_new_cont_2, t=time_test,  u=u_test_control, integrator='odeint')

x_sim_cont_2 = x_simulated_cont_2[:,0]
y_sim_cont_2 = x_simulated_cont_2[:,1]
z_sim_cont_2 = x_simulated_cont_2[:,2]

# Diff IC Diff Control Strategy
x0_new_cont_3 = np.array([2, 3, 3])
x_simulated_cont_3 = model.simulate(x0_new_cont_3, t=time_test,  u=u_test_control_2, integrator='odeint')

x_sim_cont_3 = x_simulated_cont_3[:,0]
y_sim_cont_3 = x_simulated_cont_3[:,1]
z_sim_cont_3 = x_simulated_cont_3[:,2]

trace = go.Scatter3d(
    x=x_sim_cont.flatten(), y=y_sim_cont.flatten(), z=z_sim_cont.flatten(),
    mode='lines', name='Sim Trajectory Same I.C. Same Control', line=dict(color='red')
)

layout = go.Layout(
    scene=dict(
        bgcolor='black',
         xaxis=dict(range=[-5, 5]),
         yaxis=dict(range=[-5, 5]),
         zaxis=dict(range=[0, 5])),
    margin=dict(l=0, r=0, b=0, t=0))

fig1 = go.Figure(data=[trace], layout=layout)

trace1 = go.Scatter3d(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    mode='lines', name='Training Trajectory', line=dict(color='blue'))

             #(trace) #Sim Trajectory Same I.C. Same Control
fig1.add_trace(trace1) #Training Traj
fig1.show()


# Second 3d plot w/ control actions
trace2 = go.Scatter3d(
    x=x_sim_cont_2.flatten(), y=y_sim_cont_2.flatten(), z=z_sim_cont_2.flatten(),
    mode='lines', name='Sim Trajectory Diff I.C. Same Control', line=dict(color='green'))

layout = go.Layout(
    scene=dict(
        bgcolor='black',
         xaxis=dict(range=[-5, 5]),
         yaxis=dict(range=[-5, 5]),
         zaxis=dict(range=[0, 5])),
    margin=dict(l=0, r=0, b=0, t=0))

fig2 = go.Figure(data=[trace], layout=layout)

trace3 = go.Scatter3d(
    x=x_sim_cont_3.flatten(), y=y_sim_cont_3.flatten(), z=z_sim_cont_3.flatten(),
    mode='lines', name='Sim Trajectory Diff I.C. Diff Control', line=dict(color='cyan'))

fig2 = go.Figure(data=[trace2], layout=layout)

             #(trace2) #Sim Trajectory Diff I.C. Same Control
fig2.add_trace(trace3) #Sim Trajectory Diff I.C. Diff Control
fig2.show()



# Plot New Control Strategy
plt.style.use('dark_background')

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 6))

ax1.plot(time_test, u0_test_2.flatten(), label=' New Control Signal 0', color='green')
ax1.set_ylabel('Control Signal 0')
ax1.grid(True)
ax1.legend()

ax2.plot(time_test, u1_test_2.flatten(), label='New Control Signal 1', color='red')
ax2.set_xlabel('Time')
ax2.set_ylabel('Control Signal 1')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()



---



# Notes on Noisy Data

There are mainly two ways in PySINDY to try to handle noisy signals:  

* The first is to use 'Smooth' differentiation methods `ps.SmoothedFiniteDifference()`, as used follows:

```
identity_lib = IdentityLibrary()
poly_lib = PolynomialLibrary(degree=2)
fourier_lib = FourierLibrary( include_sin=True, include_cos=True)
combined_lib = identity_lib+fourier_lib

feature_names = ['x','y','z']
opt = ps.STLSQ(threshold=0.2) #0.2

model = ps.SINDy(differentiation_method=ps.SmoothedFiniteDifference(), feature_names=feature_names, feature_library=combined_lib, optimizer=opt)
model.fit(x_train, t=dt)
model.print()
```


* The second is to operate with multiple trajectories, i.e., assuming the signal is not cleanable, one proceeds by providing several trajectories to better understand the evolution of the dynamic system despite the noise. as follow:


FROM OFFICIAL DOC:

 https://pysindy.readthedocs.io/en/latest/examples/1_feature_overview/example.html#Multiple-trajectories
```
if __name__ != "testing":
    n_trajectories = 20
    sample_range = (500, 1500)
else:
    n_trajectories = 2
    sample_range = (10, 15)
x0s = np.array([36, 48, 41]) * (np.random.rand(n_trajectories, 3) - 0.5) + np.array(
    [0, 0, 25]
)
x_train_multi = []
for i in range(n_trajectories):
    x_train_multi.append(
        solve_ivp(
            lorenz, t_train_span, x0s[i], t_eval=t_train, **integrator_keywords
        ).y.T
    )

model = ps.SINDy()
model.fit(x_train_multi, t=dt)
model.print()
```





---



Last Revision: G. Oddo 18/01/2024